# CH01.3. **Hardware Control**

## 00. **작업 환경 설정**

#### 00.0. **사전 변수 설정**

In [1]:
SEED_NUM = 2025

#### 00.1. **라이브러리 호출 및 옵션 설정**

In [ ]:
#(1) Import libraries
import os
import random
import numpy as np
import torch

#(2) Set up options
os.environ['PYTHONHASHSEED'] = str(SEED_NUM)
random.seed(a=SEED_NUM)
np.random.seed(seed=SEED_NUM)
torch.use_deterministic_algorithms(mode=True)
torch.manual_seed(seed=SEED_NUM)
torch.mps.manual_seed(seed=SEED_NUM)

#(3) Set up device
if torch.backends.mps.is_available() :
    device = torch.device(device='mps')
else :
    device = torch.device(device='cpu')
print(f'>> Device : {device}')

#### 00.2. **사용자정의함수 정의**

In [3]:
pass

#### 00.3. **클래스 정의**

In [ ]:
class ExampleModel(torch.nn.Module) :
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(in_features=3, out_features=1)
    def forward(self, x:torch.Tensor) :
        x = self.fc(x)
        return x

<b></b>

## 01. **하드웨어 제어**

#### 01.1. **텐서 하드웨어 할당** : `torch.tensor.to(···, device=device)`

In [5]:
#(1) Define `tensor_01`
tensor_01 = torch.tensor(data=[1, 2, 3], dtype=torch.float32)

#(2) Print `tensor_01`
tensor_01

tensor([1., 2., 3.])

In [6]:
#(3) Move `tensor_01` to mps device
tensor_01_mps = tensor_01.to(device='mps')

#(4) Print `tensor_01_mps`
tensor_01_mps

tensor([1., 2., 3.], device='mps:0')

#### **(`PLUS`)** 텐서들이 다른 하드웨어에 정의됐을 경우, 연산이 불가함

In [7]:
try : 
    tensor_01 + tensor_01_mps
except RuntimeError as error : 
    print(f'>> AN ERROR OCCURRED. ERROR MESSAGE IS "{error}".')

>> AN ERROR OCCURRED. ERROR MESSAGE IS "Expected all tensors to be on the same device, but found at least two devices, mps:0 and cpu!".


#### 01.2. **모델 하드웨어 할당** : `torch.nn.Module.to(···, device=device)`

In [8]:
#(1) Define `model`
model = ExampleModel()

#(2) Print `model`'s device
print(f'>> Device : {next(model.parameters()).device}')

>> Device : cpu


In [9]:
#(3) Move `model` to mps device
model = model.to(device='mps')

#(4) Print `model`'s device
print(f'>> Device : {next(model.parameters()).device}')

>> Device : mps:0


In [ ]:
#(4) Run `model` with `tensor_01_mps` as input
model(x=tensor_01_mps)

tensor([0.3756], device='mps:0', grad_fn=<LinearBackward0>)

#### **(`PLUS`)** 특정 하드웨어에서 학습하기 위해선, 입력 데이터 역시 이동시켜야함 

In [11]:
try : 
    model(x=tensor_01)
except RuntimeError as error : 
    print(f'>> AN ERROR OCCURRED. ERROR MESSAGE IS "{error}".')

>> AN ERROR OCCURRED. ERROR MESSAGE IS "Tensor for argument input is on cpu but expected on mps".
